# Julia Distributed Computing Tutorial

This tutorial demonstrates the basics of distributed computing in Julia, including:

1. **Serial computation** (baseline for comparison)
2. **Simple distributed computation** using `@distributed` macro  
3. **Advanced distributed computation** with explicit chunking
4. **Performance comparison** and scalability analysis
5. **Best practices** for distributed computing

## Key Concepts

**Distributed Computing vs Multithreading:**
- **Processes** have separate memory spaces (no shared memory)
- **Communication** happens via message passing
- **Scalability** can extend across multiple machines in a cluster  
- **Overhead** is higher but provides better isolation and fault tolerance

## Learning Objectives

By the end of this tutorial, you will understand:
- When to use distributed computing vs multithreading
- How to implement distributed algorithms in Julia
- Performance trade-offs and optimization strategies
- Best practices for distributed system design

In [ ]:
using Pkg
println("Active project: $(Pkg.project().path)")

In [ ]:
using Distributed
using Statistics
using BenchmarkTools  # For performance benchmarking

# Display system information
println("Julia Distributed Computing Tutorial")
println("=" ^ 45)
println("Available CPU cores: $(Sys.CPU_THREADS)")
println("Current Julia processes: $(nprocs())")

In [ ]:
# Add worker processes (automatically detects available cores)
println("\nAdding worker processes...")

# We are adding 10 additional processes
addprocs(10)

println("Main process ID: $(myid())")
println("Worker process IDs: $(workers())")
println("Total processes: $(nprocs())")
println("✓ Distributed environment ready!")

In [ ]:
# Load required packages on all worker processes
@everywhere begin
    using Statistics
    using BenchmarkTools
end

println("\n✓ Packages loaded on all worker processes")

In [ ]:
@everywhere begin
    """
        simulate_heavy_compute(t::Float64)::Nothing

    Simulates a computationally expensive operation by sleeping for `t` seconds.
    This represents any CPU-intensive task like numerical computation, simulation, etc.
    
    # Arguments
    - `t::Float64`: Sleep duration in seconds

    # Example
    ```julia
    simulate_heavy_compute(0.1)  # Simulates 0.1 seconds of work
    ```
    """
    function simulate_heavy_compute(t::Float64)::Nothing
        sleep(t)
        return nothing
    end

    """
        create_line(x::Float64)::Float64

    A simple linear function that computes `1.0 * x + 2.0`.
    This represents a deterministic computation that we can verify for correctness.

    # Arguments
    - `x::Float64`: Input value

    # Returns
    - `Float64`: Linear transformation of input
    """
    function create_line(x::Float64)::Float64
        return 1.0 * x + 2.0
    end

    """
        qoi(x::Float64)::Float64

    Computes the "Quantity of Interest" (QoI) for a given input.
    Combines simulated heavy computation with a deterministic calculation.
    This represents a typical scientific computing workload.
    """
    function qoi(x::Float64)::Float64
        simulate_heavy_compute(0.01)  # Reduced time for faster demo
        return create_line(x)
    end

    """
        sum_serial(xarr::AbstractVector)::Float64

    **SERIAL COMPUTATION (Single-process baseline)**

    Computes the sum of QoI for all elements using a single process.
    This serves as our baseline for performance comparison.

    # Performance Characteristics
    - Uses only one process (and one CPU core)
    - Predictable, deterministic execution
    - No inter-process communication overhead
    """
    function sum_serial(xarr::AbstractVector)::Float64
        y_serial = 0.0
        for x in xarr
            y_serial += qoi(x)
        end
        return y_serial
    end

    """
        process_chunk(chunk_indices::UnitRange{Int}, xarr::AbstractVector)::Float64

    **HELPER FUNCTION: Process a chunk of data**

    Processes a specific range of indices from the input array.
    This function runs on worker processes in the distributed computation.
    """
    function process_chunk(chunk_indices::UnitRange{Int}, xarr::AbstractVector)::Float64
        chunk_sum = 0.0
        for i in chunk_indices
            chunk_sum += qoi(xarr[i])
        end
        return chunk_sum
    end
end

println("✓ Core functions loaded on all worker processes")

In [ ]:
"""
    sum_distributed_simple(xarr::AbstractVector)::Float64

**DISTRIBUTED COMPUTATION (Simple approach)**

Computes the sum of QoI using Julia's `@distributed` macro.
This is the simplest way to distribute computation across processes.

# How it works
1. `@distributed (+)` automatically distributes loop iterations across workers
2. Each worker processes a subset of the array elements
3. Results are automatically combined using the `+` operation
4. Communication and synchronization are handled automatically

# Advantages
- Very simple to implement
- Automatic load balancing
- Built-in result aggregation

# Disadvantages  
- Less control over work distribution
- May have higher communication overhead for small chunks
"""
function sum_distributed_simple(xarr::AbstractVector)::Float64
    result = @distributed (+) for x in xarr
        qoi(x)
    end
    return result
end

"""
    sum_distributed_chunked(xarr::AbstractVector)::Float64

**DISTRIBUTED COMPUTATION (Chunked approach)**

Computes the sum of QoI using explicit work distribution with larger chunks.
This approach provides more control over how work is divided among processes.

# How it works
1. Divide the array indices into chunks, one per worker process
2. Use `pmap` to apply `process_chunk` function to each chunk in parallel
3. Each worker processes its entire chunk locally (reducing communication)
4. Combine results from all workers

# Advantages
- Better control over work distribution  
- Lower communication overhead (fewer, larger messages)
- More efficient for large arrays

# Disadvantages
- Slightly more complex to implement
- Manual load balancing required
"""
function sum_distributed_chunked(xarr::AbstractVector)::Float64
    # Handle edge case: if fewer elements than processes, use serial computation
    if length(xarr) < nprocs()
        return sum_serial(xarr)
    end
    
    # Split the array indices into chunks for each worker process
    chunks = Distributed.splitrange(1, length(xarr), nprocs())
    
    # Use pmap to distribute chunk processing across workers
    chunk_results = pmap(chunk -> process_chunk(chunk, xarr), chunks)
    
    # Sum up results from all chunks
    return sum(chunk_results)
end

println("✓ Distributed functions defined")

In [ ]:
# =============================================================================
# PERFORMANCE COMPARISON DEMONSTRATION
# =============================================================================

# Create test data
println("\n" * "=" ^ 50)
println("PERFORMANCE COMPARISON DEMONSTRATION")  
println("=" ^ 50)

println("Setting up test data...")
xarr = range(1.0, 100.0, length=100)  # Moderate size for notebook demo
println("Array size: $(length(xarr)) elements")
println("Expected result: $(sum(create_line.(xarr)))")

# Pre-compile functions (important for accurate benchmarking)
println("\nPre-compiling functions...")
_ = sum_serial(xarr[1:5])
_ = sum_distributed_simple(xarr[1:5])  
_ = sum_distributed_chunked(xarr[1:5])
println("✓ Functions compiled")


In [ ]:
# =============================================================================
# 1. SERIAL COMPUTATION (Baseline)
# =============================================================================
println("\n1. SERIAL COMPUTATION")
println("-" ^ 30)
print("Running serial computation... ")
serial_result = @timed sum_serial(xarr)
println("✓ Complete")
println("Result: $(serial_result.value)")
println("Time: $(round(serial_result.time, digits=3)) seconds")
println("Memory: $(serial_result.bytes) bytes")
println("Process: Main process only")

In [ ]:
# =============================================================================
# 2. DISTRIBUTED COMPUTATION (Simple @distributed)
# =============================================================================
println("\n2. DISTRIBUTED COMPUTATION (Simple)")
println("-" ^ 40)
println("✅ Using @distributed macro for automatic distribution")

# Run multiple times to show consistency
simple_results = Float64[]
simple_times = Float64[]

for i in 1:3
    print("Run $i... ")
    result = @timed sum_distributed_simple(xarr)
    push!(simple_results, result.value)
    push!(simple_times, result.time)
    println("Result: $(result.value), Time: $(round(result.time, digits=3))s")
end

println("Results consistency: $(length(unique(simple_results)) == 1 ? "✓ Consistent" : "✗ Inconsistent")")
println("Average time: $(round(mean(simple_times), digits=3)) seconds")
println("Processes used: Main + $(length(workers())) workers")

In [ ]:
# =============================================================================
# 3. DISTRIBUTED COMPUTATION (Chunked approach)
# =============================================================================
println("\n3. DISTRIBUTED COMPUTATION (Chunked)")
println("-" ^ 40)
println("✅ Using explicit chunking with pmap for better control")

# Run multiple times to show consistency
chunked_results = Float64[]
chunked_times = Float64[]

for i in 1:3
    print("Run $i... ")
    result = @timed sum_distributed_chunked(xarr)
    push!(chunked_results, result.value)
    push!(chunked_times, result.time)
    println("Result: $(result.value), Time: $(round(result.time, digits=3))s")
end

println("Results consistency: $(length(unique(chunked_results)) == 1 ? "✓ Consistent" : "✗ Inconsistent")")
println("Average time: $(round(mean(chunked_times), digits=3)) seconds")
println("Processes used: Main + $(length(workers())) workers")

In [ ]:
# =============================================================================
# PERFORMANCE SUMMARY & ANALYSIS
# =============================================================================
println("\n" * "=" ^ 40)
println("PERFORMANCE SUMMARY")
println("=" ^ 40)
println("Serial time:          $(round(serial_result.time, digits=3)) seconds")
println("Simple distributed:   $(round(mean(simple_times), digits=3)) seconds")
println("Chunked distributed:  $(round(mean(chunked_times), digits=3)) seconds")
println()
println("Speedup (simple):     $(round(serial_result.time / mean(simple_times), digits=2))x")
println("Speedup (chunked):    $(round(serial_result.time / mean(chunked_times), digits=2))x")
println("Efficiency (simple):  $(round(100 * serial_result.time / (mean(simple_times) * nprocs()), digits=1))%")
println("Efficiency (chunked): $(round(100 * serial_result.time / (mean(chunked_times) * nprocs()), digits=1))%")

# Accuracy check
println("\nACCURACY CHECK")
println("-" ^ 20)
expected = serial_result.value
println("Expected result:      $expected")
println("Serial result:        $(serial_result.value) ✓")
println("Simple results:       $(simple_results) $(all(r ≈ expected for r in simple_results) ? "✓" : "✗")")
println("Chunked results:      $(chunked_results) $(all(r ≈ expected for r in chunked_results) ? "✓" : "✗")")

## Distributed vs Multithreading Comparison

### Distributed Computing ✅
- **Separate memory spaces** (no shared memory issues)
- **Scalable across machines** (can use cluster resources)
- **Fault tolerant** (process failure doesn't crash others)
- **Better for large-scale, independent computations**
- ❌ Higher communication overhead
- ❌ More complex data sharing

### Multithreading ✅  
- **Lower overhead** for shared memory access
- **Faster communication** between threads
- **Better for fine-grained parallelism**
- ❌ Limited to single machine
- ❌ Potential race conditions with shared data
- ❌ One thread crash can affect the whole process

### When to Choose Each
- **Use Distributed**: Large datasets, independent tasks, cluster computing
- **Use Multithreading**: Shared memory algorithms, fine-grained parallelism, single machine

In [ ]:
# =============================================================================
# KEY TAKEAWAYS
# =============================================================================
println("\n" * "=" ^ 40)
println("KEY TAKEAWAYS")
println("=" ^ 40)
println("1. Distributed computing is ideal for large-scale, independent tasks")
println("2. Use @distributed for simple cases, pmap for more control")
println("3. Chunking reduces communication overhead and improves efficiency")
println("4. Consider communication costs vs computation costs")
println("5. Distributed computing scales beyond single-machine limitations")
println("6. Always verify correctness across all processes")
println()
println("To add more processes: julia -p N script.jl or addprocs(N)")

# Clean up worker processes (optional - good practice)
println("\nCleaning up worker processes...")
rmprocs(workers())
println("✓ Worker processes removed")
println("Current processes: $(nprocs())")